### PROYECTO 1 - ETAPA 1
**GRUPO 2:** \
Juana Mejía 202021512\
Daniela Espinosa 202022615 \
Panblo Ortega

#### Carga de datos

In [ ]:
import pandas as pd
df = pd.read_excel('ODScat_345.xlsx')
df.shape

#### Perfilamiento de los Datos


In [ ]:
# Estadisticas descriptivas
df.describe()

El describe nos da información sobre la única variable numérica, sin embargo, debemos recordar que esta variable es una categoria codificada. Por este motivo la media, desviación y percentiles no resultan muy útiles. El count nos dice el número de valores que tiene la columna.

In [ ]:
# Informacion del data set
df.info()

Como podemos ver, el dataframe no tiene valores nulos.

In [ ]:
# Análisis de duplicados
duplicates = df.duplicated().sum()
print(f"\nNúmero de registros duplicados: {duplicates}")

Tampoco tiene duplicados.

In [ ]:
levels = df.apply(pd.Series.unique)
num_levels = df.apply(pd.Series.nunique)

print(levels)
print(num_levels)

La columna sdg contiene valores numéricos y tiene tres ategorías: [3, 4, 5]
Todos los valores de Textos_espanol son de texto y son diferentes.

#### Limpieza de datos

In [22]:
data = df.copy()

In [ ]:
# Normalización de textos
data['Textos_espanol'] = data['Textos_espanol'].str.lower()
data['Textos_espanol'] = data['Textos_espanol'].str.replace('[^\w\s]', '', regex=True)

#### Vectorización de textos
Encontrar el número óptimo del máximo de features:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, accuracy_score
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

# Cargar las palabras de parada en español
spanish_stopwords = stopwords.words('spanish')

# Definir el pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=spanish_stopwords)),
    ('clf', MultinomialNB())
])

# Definir el rango de parámetros para buscar
param_grid = {
    'tfidf__max_features': [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
}

# Configurar el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=make_scorer(accuracy_score))

# Ajustar el modelo con el GridSearchCV
grid_search.fit(data['Textos_espanol'], data['sdg'])

# Mejor número de max_features
best_max_features = grid_search.best_params_['tfidf__max_features']
print(f"Mejor número de max_features: {best_max_features}")

# Obtener los resultados de precisión media para cada valor de max_features
mean_scores = grid_search.cv_results_['mean_test_score']

# Graficar los resultados
plt.figure(figsize=(10, 6))
plt.plot(param_grid['tfidf__max_features'], mean_scores, marker='o', linestyle='-')
plt.xlabel('Número de max_features')
plt.ylabel('Precisión media')
plt.title('Rendimiento del modelo según max_features')
plt.grid(True)
plt.show()



Se utilizó cross-validation para elegir el número optimo de características del modelo después de vectorizar.

**Vectorizar el texto**

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar las palabras de parada en español
spanish_stopwords = stopwords.words('spanish')

# Vectorización de textos
vectorizer = TfidfVectorizer(stop_words=spanish_stopwords, max_features=best_max_features)
text_vectors = vectorizer.fit_transform(data['Textos_espanol'])
text_data = pd.DataFrame(text_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenar con el dataset original
data = pd.concat([data.drop(columns=['Textos_espanol']), text_data], axis=1)
data

Se vectorizó el texto con base en la cantidad óptima de características obtenidas anteriormente. El resultado es un dataframe con la misma cantidad de filas pero 8000 columnas de características, una por palabra. Cada una de estas columnas es un vector que depende de la relevancia de la palabra.

#### Separar los datos en train y test

In [26]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en características y variable objetivo
X = data.drop(columns=['sdg'])
Y = data[['sdg']]


mapping = {3: 0, 4: 1, 5: 2}
Y = Y['sdg'].map(mapping)


# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


Separación de los datos en variable objetivo y características y mapping de las categorias (3,4,5) a (0,1,2) respectivamente para poder utilizar XGboost.

#### Implementación de modelos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Definir los modelos en un diccionario
models = {
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(),
    'XGboost': XGBClassifier(),
    'NeuralNet': MLPClassifier()
}

# Recorrer los modelos
for name, model in models.items():
       
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predecir con el modelo entrenado
    y_pred = model.predict(X_test)
    
    # Evaluar el modelo
    print(f"\n{name}:")
    print(classification_report(y_test, y_pred))

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['sdg3', 'sdg4', 'sdg5'], yticklabels=['sdg3', 'sdg4', 'sdg5'])
    plt.title(f'{name} - Confusion Matrix')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()
    


Se implementaron 5 modelos diferentes, por ahora los que más prometen son Naive Bayes y la red neuronal.

### Construcción del modelo con optimización de hiperparámetros ###

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV



# Definir los hiperparámetros a optimizar para cada modelo
param_grids = {
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski','euclidean', 'manhattan']
    },
    'Naive Bayes': {
        'alpha': [0.1, 0.5, 1.0, 2.0]
    },
    'Random Forest': {
        'criterion':['gini', 'entropy'],
        'n_estimators': [50, 100, 200],
        'max_depth': [1, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    },
    'XGboost': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    },
    'NeuralNet': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['tanh', 'relu'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    }
}



# Recorrer los modelos y realizar la búsqueda de hiperparámetros
for name, model in models.items():
    
    # Seleccionar el tipo de búsqueda de hiperparámetros (GridSearchCV o RandomizedSearchCV)
    if name in ['KNN', 'Naive Bayes']:
        search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1)
    else:
        search = RandomizedSearchCV(model, param_grids[name], n_iter=15, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
    
    # Entrenar el modelo con búsqueda de hiperparámetros
    search.fit(X_train, y_train)
    
    # Mejor modelo encontrado
    best_model = search.best_estimator_
    
    # Predecir con el mejor modelo
    y_pred = best_model.predict(X_test)
    
    # Evaluar el mejor modelo
    print(f"\n{name} Classifier Report (Best Model):")
    print(classification_report(y_test, y_pred))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['sdg3', 'sdg4', 'sdg5'], yticklabels=['sdg3', 'sdg4', 'sdg5'])
    plt.title(f'{name} - Confusion Matrix (Best Model)')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()
    
    # Mostrar los mejores hiperparámetros
    print(f"Best hyperparameters for {name}: {search.best_params_}")

    print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred)) 


Utilizando GridSearch y RandomizedSearch para la optimización de hiperparámetros se lograron mejorar las métricas de calidad de los modelos. El mejor modelo resultante es la red neuronal y en segundo lugar Naive Bayes.

**Modelos con hiperparametros dados:**

Esto se hizo en caso de perder el proceso de optimización de la parte superior, ya que se demoró horas en realizar el cross-validation para todos los modelos.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
# Definir los modelos en un diccionario
models = {'KNN': KNeighborsClassifier(n_neighbors= 9,weights='distance',metric='minkowski'),
    'Naive Bayes': MultinomialNB(alpha= 1.0),
    'Random Forest': RandomForestClassifier(n_estimators= 200, criterion = 'entropy',max_depth= 20, min_samples_split= 5, min_samples_leaf= 2, bootstrap= False),
    'XGboost': XGBClassifier(subsample=1.0, n_estimators= 200, min_child_weight= 1, max_depth= 5, learning_rate= 0.1, colsample_bytree= 1.0),
    'NeuralNet': MLPClassifier(solver= 'adam',  alpha= 0.01,learning_rate='constant', hidden_layer_sizes= (100,),activation= 'tanh')}

# Recorrer los modelos
for name, model in models.items():
       
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predecir con el modelo entrenado
    y_pred = model.predict(X_test)
    
    # Evaluar el modelo
    print(f"\n{name}:")
    report = classification_report(y_test, y_pred, target_names=['sdg3', 'sdg4', 'sdg5'], output_dict=True)

# Convertir el reporte a un DataFrame para mejor manipulación
    report_df = pd.DataFrame(report).transpose()
# Formatear el DataFrame para mostrar 4 cifras decimales
    report_df = report_df.round(4)
    print(report_df)

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['sdg3', 'sdg4', 'sdg5'], yticklabels=['sdg3', 'sdg4', 'sdg5'])
    plt.title(f'{name} - Confusion Matrix')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Datos
labels = ['KNN', 'Naive Bayes', 'Random Forest', 'XGBoost', 'ANN']
f1_scores = [95.92, 97.53, 96.55, 95.19, 97.65]

# Configuración del gráfico
x = np.arange(len(labels))  # Etiquetas para cada algoritmo
width = 0.4  # Ancho de las barras

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(10, 6))

# Graficar las barras para F1-score
rects1 = ax.bar(x, f1_scores, width, label='F1-score', color='#a21942')

# Añadir etiquetas y título con tamaños de fuente aumentados
ax.set_xlabel('Algoritmo', fontsize=14)
ax.set_ylabel('F1-score (%)', fontsize=14)
ax.set_title('Comparación de F1-score por Algoritmo', fontsize=16)

# Personalizar los ticks del eje x y del eje y
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=12)
ax.tick_params(axis='y', labelsize=12)

# Limitar el eje y entre 90 y 100
ax.set_ylim(90, 100)

# Añadir los valores encima de cada barra
for rect in rects1:
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2.,  # Posición en el eje x
        height,  # Altura (posición en el eje y)
        f'{height:.2f}',  # Formato del texto (F1-score con 2 decimales)
        ha='center', va='bottom', fontsize=12  # Aumentar tamaño del texto sobre las barras
    )

# Mostrar gráfico
plt.tight_layout()
plt.show()


Como se puede observar el mejor algoritmo es ANN con un f1-score de 97.65. Sin embargo, en el momento de hacer la recomendación al cliente se preferiría utilizar Naive Bayes debido a que es un modelo más eficiente computacionalmente y fácil de optimizar debido a que tiene un solo hiperparámetro. Adicionalmente es un modelo interpretable a diferencia de la red neuronal.

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

model=MultinomialNB(alpha= 1.0)


# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred = model.predict(X_test)


print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['sdg3', 'sdg4', 'sdg5'], yticklabels=['sdg3', 'sdg4', 'sdg5'])
plt.title('NB - Confusion Matrix')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.show()


# Interpretabilidad
feature_log_probs = model.feature_log_prob_

top_n = 3
for i, class_label in enumerate(model.classes_):
    sorted_idx = np.argsort(feature_log_probs[i])[::-1]
    print("-------------------------------------------------")
    print(f"Top {top_n} palabras más importantes para el objetivo de desarrollo sostenible {class_label+3}:")
    for idx in sorted_idx[:top_n]:
        print(f"{X_train.columns[idx]}: {np.exp(feature_log_probs[i][idx]):.4f}")



Se puede interpretar el modelo para extraer las palabras que tienen más relevancia para el modelo en la toma de decisiones. Como se puede observar se ajustan muy bien a cada ODS.